In [2]:
 # Step 2: Import Libraries
# import sys
# !{sys.executable} -m pip install ta


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from ta.momentum import RSIIndicator
from ta.trend import EMAIndicator, MACD



In [4]:
df = yf.download('IEX', start='2015-01-01', end='2025-06-30')
df = df[['Close']].copy()
df.reset_index(inplace=True)
df.columns = ['Date', 'Close']


C:\Users\2000s\AppData\Local\Temp\ipykernel_11624\1021479335.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('IEX', start='2015-01-01', end='2025-06-30')
[*********************100%***********************]  1 of 1 completed


In [5]:
df['EMA_20'] = EMAIndicator(df['Close'], window=20).ema_indicator()
df['RSI'] = RSIIndicator(df['Close'], window=14).rsi()
df['MACD'] = MACD(df['Close']).macd()
df.dropna(inplace=True)



In [6]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['Close', 'EMA_20', 'RSI', 'MACD']])


In [7]:
x_train, y_train = [], []

for i in range(60, len(scaled_data)):
    x_train.append(scaled_data[i-60:i])
    y_train.append(scaled_data[i, 0])  # 'Close' price

x_train, y_train = np.array(x_train), np.array(y_train)


In [8]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 4)))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=10, batch_size=32)


C:\Users\2000s\tfenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 0.0243
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 6.2740e-04
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 5.2218e-04
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 5.2175e-04
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 6.1471e-04
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 4.8729e-04
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 4.3933e-04
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 3.9719e-04
Epoch 9/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - loss: 3.6547e-04
Epoch 10/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 3.6495e-04


In [10]:
 model.save('keras_model.h5')

In [ ]:
past_60_days = scaled_data[-60:]
future_input = past_60_days.reshape(1, 60, 4)


In [ ]:
future_prices = []
input_seq = past_60_days.copy()

for _ in range(30):
    pred = model.predict(input_seq.reshape(1, 60, 4), verbose=0)
    future_prices.append(pred[0][0])

    # Fill the remaining 3 features (EMA, RSI, MACD) with previous values
    next_input = [pred[0][0], input_seq[-1][1], input_seq[-1][2], input_seq[-1][3]]
    input_seq = np.append(input_seq[1:], [next_input], axis=0)

# Inverse transform predictions
dummy = np.zeros((30, 4))
dummy[:, 0] = future_prices
predicted_prices_30_days = scaler.inverse_transform(dummy)[:, 0]


In [ ]:
last_date = df['Date'].iloc[-1]
future_dates = [last_date + datetime.timedelta(days=i) for i in range(1, 31)]


In [ ]:
train_predictions = model.predict(x_train, verbose=0)
train_plot = np.zeros((len(scaled_data), 1))
train_plot[60:] = train_predictions

train_plot = scaler.inverse_transform(np.hstack([train_plot, scaled_data[:, 1:]]))[:, 0]
original_prices = df['Close'].values

plt.figure(figsize=(14, 6))
plt.plot(df['Date'], original_prices, label='Original Price')
plt.plot(df['Date'], train_plot, label='Predicted (Train)', linestyle='--')
plt.legend()
plt.title('Original Price vs Predicted Price (Train)')
plt.xlabel('Date')
plt.ylabel('Price (INR)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(future_dates, predicted_prices_30_days, marker='o', label='Predicted Price (Next 30 days)')
plt.title('Stock Price Forecast for Next 30 Days')
plt.xlabel('Date')
plt.ylabel('Price (INR)')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
for date, price in zip(future_dates, predicted_prices_30_days):
    print(f"{date.strftime('%Y-%m-%d')} → ₹{round(price, 2)}")
